In [1]:
# MC, TD는 MDP를 모를 때 value를 평가하기 위한 방법론

import random

class GridWorld():
    def __init__(self):
        # x, y 좌표 0으로 초기화
        self.x = 0
        self.y = 0

    def step(self, a): # 에이전트로부터 액션을 받아서 상태 변이를 일으키고, 보상을 정해주는 함수
        if a == 0:
            self.move_right()
        elif a == 1:
            self.move_left()
        elif a == 2:
            self.move_up()
        elif a == 3:
            self.move_down()

        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done
    
    def move_right(self):
        self.x += 1
        if self.x > 3:
            self.x = 3 # 4 x 4 gridworld 이므로

    def move_left(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0

    def move_up(self):
        self.y += 1
        if self.y > 3:
            self.y = 3

    def move_down(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0

    def is_done(self):
        if self.x == 3 and self.y == 3: # Terminal State
            return True
        else:
            return False
        
    def get_state(self): # 현재 위치 반환
        return (self.x, self.y)
    
    def reset(self): # 초기화 (현재 MC를 구현하기 때문에 episode가 끝날때마다 업데이트 후 reset될 것임을 유추할 수 있음)
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [2]:
class Agent():
    def __init__(self):
        pass

    def select_action(self):
        coin = random.random() # 0~1 사이의 난수 생성
        if coin < 0.25:
            action = 0
        
        elif coin < 0.5:
            action = 1

        elif coin < 0.75:
            action = 2

        else :
            action = 3

        return action

In [3]:
# Monte Carlo (Calculate Value Function)
def main():
    env = GridWorld()
    agent = Agent()
    data = [[0, 0, 0, 0], [0, 0, 0, 0], # Value Function 값 모두 0으로 초기화
            [0, 0, 0, 0], [0, 0, 0, 0]]
    gamma = 1.0
    alpha = 0.0001

    for k in range(50000): # 총 50,000번의 episode를 돌림
        done = False
        history = []
        while not done:
            action = agent.select_action()
            (x, y), reward, done = env.step(action)
            history.append((x, y, reward))
            # print("history : ",history)
        env.reset()
    # history = [(0, 0, -1), (0, 1, -1), (0, 2, -1), (0, 3, -1), (1, 3, -1), (2, 3, -1), (3, 3, -1)]
    # history가 이런식으로 찍히는 이유는, cumulative reward가 아니라 그냥 매 step에서 reward를 찍어주기 때문임

        # print("history : ", history)
        cum_reward = 0
        for transition in history[::-1]:
            x, y, reward = transition
            data[x][y] = data[x][y] + alpha*(cum_reward - data[x][y]) # history를 거꾸로 돌면서 value function을 업데이트 (history의 각 state마다의 cum_reward(G_t)를 구해서 value function을 업데이트)
            cum_reward = reward + gamma*cum_reward # 이 코드가 value func업데이트 하는 코드보다 늦게 나오는 이유는, terminal state에서의 value function값은 0이므로 무의미한 value function을 업데이트 후 cum_reward 업데이트
            # print("transition : ", transition)
            # print("data : ", data)
            # print("cum_reward : ", cum_reward)
            
    for row in data:
        print(row)

if __name__ == "__main__":
    main()

[-60.87716763352612, -59.08418041758082, -56.19236396740267, -53.859105180502]
[-58.1671879283501, -55.05174633692308, -50.408488837922384, -45.65857318098117]
[-53.87442073574309, -49.7930516326734, -40.770664527556896, -30.081552754275908]
[-51.053096584554865, -45.572042572992196, -30.47886009102835, 0.0]


In [4]:
# Temporal Difference (Calculate Value Function)
def main():
    env = GridWorld()
    agent = Agent()
    data = [[0, 0, 0, 0], [0, 0, 0, 0], 
            [0, 0, 0, 0], [0, 0, 0, 0]]
    gamma = 1.0
    alpha = 0.01

    for k in range(50000):
        done = False
        while not done:
            x, y = env.get_state()
            action = agent.select_action()
            (x_prime, y_prime), reward, done = env.step(action)
            x_prime, y_prime = env.get_state()
            
            data[x][y] = data[x][y] + alpha*(reward + gamma*data[x_prime][y_prime] - data[x][y]) # 한 번의 액션마다 데이터 테이블이 업데이트
        env.reset()

    for row in data:
        print(row)
    
if __name__ == "__main__":
    main()

[-58.96951907410966, -56.86197385282995, -53.443384357892924, -51.015835233922886]
[-57.020211914645266, -53.833517279262665, -48.808948960588886, -43.670815728700006]
[-53.7422739198754, -48.98724753449541, -40.892303575403844, -30.785899012021204]
[-51.12245178133139, -44.04770236235922, -27.220485794317447, 0]
